# Libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

# --------------------------------

In [4]:
input_shape = (4, 28, 28, 3)

In [5]:
x = tf.random.normal(input_shape)

In [11]:
y = tf.keras.layers.Conv2D(
    2,
    3,
    activation = "relu",
    input_shape = input_shape[1:]
)(x)

TensorShape([4, 26, 26, 2])